<a href="https://colab.research.google.com/github/sharma18yash/ire_project/blob/main/MLNS_Siamese_dislitBert_extended_old_split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install torchmetrics
!pip install pytorch_lightning
!pip install wandb
!wandb.login()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 4.9 MB/s 
     |████████████████████████████████| 7.6 MB 47.0 MB/s 
     |████████████████████████████████| 163 kB 75.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 529 kB 5.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 796 kB 5.2 MB/s 
     |████████████████████████████████| 136 kB 56.8 MB/s 
     |████████████████████████████████| 87 kB 5.1 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115942 sha256=89f40bcc8b8f63c4b04e618ce61ed225c2406cec1992f8a5c3a87a0b6c25

In [ ]:
import json

In [ ]:
import json
from functools import partial

#from transformers import XLMRobertaTokenizer, XLMRobertaModel
from transformers import AutoTokenizer, AutoModel
import torch
from torch import nn
from torch.optim import AdamW
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

import torchmetrics
import pytorch_lightning as pl

from pytorch_lightning.loggers import WandbLogger

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-multilingual-cased')
encoder = AutoModel.from_pretrained('distilbert-base-multilingual-cased')
#tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
#encoder = XLMRobertaModel.from_pretrained('xlm-roberta-base')
encoder

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(

In [ ]:
EXP_NAME = 'mlns-distilbert-regressor_old_split'
TRAIN_BATCH_SIZE = 16 #4 #8 #16
DEV_BATCH_SIZE = 32 #64
ACCUMULATE_GRAD = 2 #8 #4 #2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Model

In [ ]:
class SimilarityRegressor(nn.Module):
    def __init__(self, encoder, embed_size=768, hidden_size=256):
        super(SimilarityRegressor, self).__init__()

        self.encoder = encoder
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        
        self.linear1 = nn.Linear(self.embed_size, self.hidden_size)
        self.activation1 = nn.LeakyReLU(negative_slope=0.1)
        self.dropout1 = nn.Dropout(p=0.2)
        self.linear2 = nn.Linear(2*self.hidden_size, self.hidden_size//2)
        self.dropout2 = nn.Dropout(p=0.2)
        self.activation2 = nn.LeakyReLU(negative_slope=0.1)
        self.linear3 = nn.Linear(self.hidden_size//2, 1)
        self.activation3 = nn.Sigmoid()

    def common_compute(self, x):
        x = self.encoder(**x)[0][:, 0]
        x = self.linear1(x)
        x = self.activation1(x)
        x = self.dropout1(x)

        return x
    
    def forward(self, x1, x2):
        x1 = self.common_compute(x1)
        x2 = self.common_compute(x2)
        x = torch.cat([torch.abs(x1 - x2), (x1 + x2)], dim=-1)
        x = self.linear2(x)
        x = self.activation2(x)
        x = self.dropout2(x)
        x = self.linear3(x)
        x = 3*self.activation3(x) + 1

        return x

In [ ]:
class LitSimilarityRegressor(pl.LightningModule):
    def __init__(self, encoder, embed_size=768, hidden_size=256):
        super(LitSimilarityRegressor, self).__init__()
        self.model = SimilarityRegressor(encoder, embed_size=embed_size, hidden_size=hidden_size)

        self.train_loss = torchmetrics.MeanMetric(compute_on_step=True)
        self.dev_loss = torchmetrics.MeanMetric(compute_on_step=False)
        self.test_loss = torchmetrics.MeanMetric(compute_on_step=False)

        self.train_mape = torchmetrics.MeanAbsolutePercentageError(compute_on_step=True)
        self.dev_mape = torchmetrics.MeanAbsolutePercentageError(compute_on_step=False)
        self.test_mape = torchmetrics.MeanAbsolutePercentageError(compute_on_step=False)

        self.train_pcc = torchmetrics.PearsonCorrCoef(compute_on_step=True)
        self.dev_pcc = torchmetrics.PearsonCorrCoef(compute_on_step=False)
        self.test_pcc = torchmetrics.PearsonCorrCoef(compute_on_step=False)

    def forward(self, x1, x2):
        return self.model(x1, x2)

    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=1e-5, betas=(0.9, 0.99), eps=1e-8, weight_decay=0.01)

    def training_step(self, batch, batch_idx):
        x1, x2, scores = batch
        output = self(x1, x2)
        loss = F.mse_loss(input=output, target=scores)

        return {'loss': loss, 'preds': output, 'target': scores}

    def validation_step(self, batch, batch_idx):
        x1, x2, scores = batch
        output = self(x1, x2)
        loss = F.mse_loss(input=output, target=scores)

        return {'loss': loss, 'preds': output, 'target': scores}
    
    def test_step(self, batch, batch_idx):
        x1, x2, scores = batch
        output = self(x1, x2)
        loss = F.mse_loss(input=output, target=scores)

        return {'loss': loss, 'preds': output, 'target': scores}

    def predict_step(self, batch, batch_idx):
        x1, x2, _ = batch
        output = self(x1, x2)

        return {'preds': output}

    def training_step_end(self, outs):
        loss = outs['loss']
        preds = outs['preds']
        target = outs['target']

        self.log('train/step/loss', self.train_loss(loss))
        self.log('train/step/mape', self.train_mape(preds, target))
        self.log('train/step/pcc', self.train_pcc(torch.reshape(preds, (-1,)), torch.reshape(target, (-1,))))

    def validation_step_end(self, outs):
        loss = outs['loss']
        preds = outs['preds']
        target = outs['target']

        self.dev_loss(loss)
        self.dev_mape(preds, target)
        self.dev_pcc(torch.reshape(preds, (-1,)), torch.reshape(target, (-1,)))

    def test_step_end(self, outs):
        loss = outs['loss']
        preds = outs['preds']
        target = outs['target']

        self.test_loss(loss)
        self.test_mape(preds, target)
        self.test_pcc(torch.reshape(preds, (-1,)), torch.reshape(target, (-1,)))

    def training_epoch_end(self, outs):
        self.log('train/epoch/loss', self.train_loss)
        self.log('train/epoch/mape', self.train_mape)
        self.log('train/epoch/pcc', self.train_pcc)

    def validation_epoch_end(self, outs):
        self.log('dev/loss', self.dev_loss)
        self.log('dev/mape', self.dev_mape)
        self.log('dev/pcc', self.dev_pcc)
    
    def test_epoch_end(self, outs):
        self.log('dev/loss', self.test_loss)
        self.log('dev/mape', self.test_mape)
        self.log('dev/pcc', self.test_pcc)

In [ ]:
model = LitSimilarityRegressor(encoder, embed_size=768, hidden_size=256)
model

LitSimilarityRegressor(
  (model): SimilarityRegressor(
    (encoder): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(119547, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0): TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out_features=768, bias=True)
            )
            (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (ffn): FFN(
     

#Data

In [ ]:
root_path = '/content/drive/MyDrive/Colab Notebooks/ire project/'
train_data = json.load(open('/content/drive/MyDrive/Colab Notebooks/ire project/train_data.json', 'r'))
test_data = json.load(open('/content/drive/MyDrive/Colab Notebooks/ire project/test_data.json', 'r'))

In [ ]:
train_data[123]

{'pair_id': '1538841592_1484040205',
 'title_1': 'Trump blames Iran after Iraqi militia storms US embassy in Baghdad over airstrikes',
 'title_2': 'Iraq: Iran attacks US as Trump threatens repercussion',
 'text_1': 'Text Size: A- A+\n\nDubai: Dozens of Iraqi militiamen and their supporters stormed the U.S. Embassy complex in Baghdad on Tuesday to protest deadly U.S. airstrikes against their Iranian-backed force, as President Donald Trump blamed Tehran.\n\nThe Pentagon dispatched two AH-64 Apache helicopters to fly over the embassy in a show of force, and about 100 Marines already in the region will be sent to the compound to reinforce its defenses, a U.S. official said.\n\n“We have taken appropriate force protection actions to ensure the safety of American citizens, military personnel and diplomats in-country, and to ensure our right of self-defense,” Defense Secretary Mark Esper said in a statement.\n\nThe anti-American protesters remained outside the embassy compound into the night i

In [ ]:
import re
def clean_text(str):
  str = re.sub("[^.0-9A-Za-z ]", "" , str)
  str = re.sub(' +',' ',str)
  str = str.strip()
  return str.lower();

In [ ]:
x = clean_text(train_data[123]['text_1'])
y = clean_text(train_data[123]['text_2'])
x,y

('text size a adubai dozens of iraqi militiamen and their supporters stormed the u.s. embassy complex in baghdad on tuesday to protest deadly u.s. airstrikes against their iranianbacked force as president donald trump blamed tehran.the pentagon dispatched two ah64 apache helicopters to fly over the embassy in a show of force and about 100 marines already in the region will be sent to the compound to reinforce its defenses a u.s. official said.we have taken appropriate force protection actions to ensure the safety of american citizens military personnel and diplomats incountry and to ensure our right of selfdefense defense secretary mark esper said in a statement.the antiamerican protesters remained outside the embassy compound into the night in baghdad as some set up tents suggesting they planned an extended sitin al jazeera reported.earlier guards lobbed tear gas and opened fire to quell the unrest which was quickly contained though dozens of protesters remained outside the compound. 

In [ ]:
# from sklearn.model_selection import train_test_split
# total_data = train_data + test_data
# train_dataset , temp_data = train_test_split(total_data,test_size = 0.3,shuffle = True)
# dev_dataset , test_dataset = train_test_split(temp_data,test_size = 0.5,shuffle = True)

# len(total_data),len(train_dataset),len(dev_dataset),len(test_dataset)

(7173, 5021, 1076, 1076)

In [ ]:
len(test_data) , len(train_data) , len(train_data)*0.8

(4316, 2857, 2285.6)

In [ ]:
train_dataset = train_data[:2285]
dev_dataset = train_data[2285:]
test_dataset = test_data

In [ ]:
def collate_fn(batch, tokenizer):
    texts_1, texts_2, scores = list(), list(), list()
    for sample in batch:

        text1 = str(sample['text_1']).lower().strip()
        text2 = str(sample['text_2']).lower().strip()
        
        score = torch.tensor([sample['score']])
        texts_1.append(text1)
        texts_2.append(text2)
        scores.append(score)

    texts_1 = tokenizer(texts_1, truncation=True, padding=True, return_tensors='pt')
    texts_2 = tokenizer(texts_2, truncation=True, padding=True, return_tensors='pt')
    scores = torch.cat(scores, dim=0).unsqueeze(1)

    return texts_1, texts_2, scores

In [ ]:
collate_partial = partial(collate_fn, tokenizer=tokenizer)
dataloader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True, collate_fn=collate_partial)
next(iter(dataloader))

({'input_ids': tensor([[  101, 26794, 10741,  ...,     0,     0,     0],
         [  101, 13735, 21484,  ...,     0,     0,     0],
         [  101, 93160, 61571,  ..., 10110, 20886,   102],
         ...,
         [  101, 48201, 47125,  ..., 32952, 12355,   102],
         [  101, 38171,   131,  ..., 10474, 11121,   102],
         [  101, 49661, 10632,  ...,   117, 24677,   102]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1]])},
 {'input_ids': tensor([[   101,    169,  10270,  ...,      0,      0,      0],
         [   101, 101328,  92431,  ...,      0,      0,      0],
         [   101,  69114,    206,  ...,      0,      0,      0],
         ...,
         [   101,  10105,  97764,  ...,  10108,  12976,    102],
         [   101,  10271,  10894,  ...,      0,      0,      0],
         [   101,  

In [ ]:
class MLNSDataModule(pl.LightningDataModule):
    def __init__(self, train_dataset, dev_dataset, test_dataset, train_batch_size, dev_batch_size, collate_fn, tokenizer):
        super(MLNSDataModule, self).__init__()
        self.train_dataset = train_dataset
        self.dev_dataset = dev_dataset
        self.test_dataset = test_dataset
        self.train_batch_size = train_batch_size
        self.dev_batch_size = dev_batch_size
        self.collate_fn = collate_fn
        self.tokenizer = tokenizer

    def train_dataloader(self):
        collate_partial = partial(self.collate_fn, tokenizer=self.tokenizer)
        return DataLoader(self.train_dataset, shuffle=True, batch_size=self.train_batch_size, collate_fn=collate_partial)

    def val_dataloader(self):
        collate_partial = partial(self.collate_fn, tokenizer=self.tokenizer)
        return DataLoader(self.dev_dataset, shuffle=False, batch_size=self.dev_batch_size, collate_fn=collate_partial)

    def test_dataloader(self):
        collate_partial = partial(self.collate_fn, tokenizer=self.tokenizer)
        return DataLoader(self.test_dataset, shuffle=False, batch_size=self.dev_batch_size, collate_fn=collate_partial)

    def predict_dataloader(self):
        #return DataLoader(self.test_dataset, shuffle=False, batch_size=self.dev_batch_size, collate_fn=collate_partial)
        return self.test_dataloader()

In [ ]:
#data_module = MLNSDataModule(train_dataset, dev_dataset, test_dataset, TRAIN_BATCH_SIZE, DEV_BATCH_SIZE, collate_fn=collate_fn, tokenizer=tokenizer)
data_module = MLNSDataModule(train_dataset, dev_dataset, test_dataset, TRAIN_BATCH_SIZE, DEV_BATCH_SIZE, collate_fn=collate_fn, tokenizer=tokenizer)
data_module

#Training

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath='/content/drive/MyDrive/Colab Notebooks/ire project/',
    filename='old_split{epoch}-{step}',
    monitor='dev/pcc',
    mode='max',
    save_top_k=5,
    verbose=True,
    save_last=False,
    save_weights_only=False,
    every_n_epochs=1
)

In [ ]:
logger = WandbLogger(save_dir=EXP_NAME, project=EXP_NAME, log_model=False)
logger

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: WARNING Path mlns-distilbert-regressor_old_split/wandb/ wasn't writable, using system temp directory.


In [ ]:
trainer = pl.Trainer(max_epochs=10,
                     accumulate_grad_batches=ACCUMULATE_GRAD,
                     accelerator='gpu',
                     check_val_every_n_epoch=1, val_check_interval=0.25,
                     enable_progress_bar=True,
                     gradient_clip_val=0.25, track_grad_norm=2,
                     enable_checkpointing=True,
                     callbacks=[checkpoint_callback],
                     logger=logger,
                     enable_model_summary=True)
                    

trainer

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model, datamodule=data_module)
# trainer.fit(model, datamodule=data_module, ckpt_path = '/content/drive/MyDrive/Colab Notebooks/ire project/epoch=0-step=178.ckpt')

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:606: UserWarning: Checkpoint directory /content/drive/MyDrive/Colab Notebooks/ire project exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type                        | Params
-----------------------------------------------------------
0 | model      | SimilarityRegressor         | 134 M 
1 | train_loss | MeanMetric                  | 0     
2 | dev_loss   | MeanMetric                  | 0     
3 | test_loss  | MeanMetric                  | 0     
4 | train_mape | MeanAbsolutePercentageError | 0     
5 | dev_mape   | MeanAbsolutePercentageError | 0     
6 | test_mape  | MeanAbsolutePercentageError | 0     
7 | train_pcc  | PearsonCorrCoef             | 0     
8 | dev_pcc    | PearsonCorrCoef            

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 17: 'dev/pcc' reached 0.08919 (best 0.08919), saving model to '/content/drive/MyDrive/Colab Notebooks/ire project/old_splitepoch=0-step=17.ckpt' as top 5


...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 9, global step 683: 'dev/pcc' was not in top 5


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 9, global step 700: 'dev/pcc' was not in top 5


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 9, global step 718: 'dev/pcc' was not in top 5
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [ ]:
test_pred = trainer.predict(datamodule=data_module)
len(test_pred)  

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:133: UserWarning: `.predict(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.predict(ckpt_path='best')` to use the best model or `.predict(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  + f" You can pass `.{fn}(ckpt_path='best')` to use the best model or"
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at mlns-distilbert-regressor_4_10_test/mlns-distilbert-regressor_4_10_test/369qni79/checkpoints/epoch=9-step=720.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at mlns-distilbert-regressor_4_10_test/mlns-distilbert-regressor_4_10_test/369qni79/checkpoints/epoch=9-step=720.ckpt


Predicting: 572it [00:00, ?it/s]

540

In [ ]:
test_result = trainer.test(model,datamodule=data_module)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        dev/loss             1.797494649887085
        dev/mape            0.7265227437019348
         dev/pcc            0.2125272899866104
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [ ]:
test_result_1 = trainer.test(model,datamodule=data_module,ckpt_path = '/content/drive/MyDrive/Colab Notebooks/ire project/old_splitepoch=6-step=502.ckpt')

INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/drive/MyDrive/Colab Notebooks/ire project/old_splitepoch=6-step=502.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at /content/drive/MyDrive/Colab Notebooks/ire project/old_splitepoch=6-step=502.ckpt


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        dev/loss            1.6380831003189087
        dev/mape            0.6706548929214478
         dev/pcc            0.2215994894504547
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [ ]:
# torch.save(trainer, '/content/drive/MyDrive/Colab Notebooks/ire project/DislitBert.pt')
# torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/ire project/DislitBert.pth')

In [ ]:
# trainer_loaded = torch.load('/content/drive/MyDrive/IRE project/DislitBert.pt')

In [ ]:
# test_result = trainer_loaded.test(datamodule=data_module)